# Update Data
**Filename:** UpdateData.ipynb <br>
**Author:** Laura Kaufmann <br>
**Purpose:** To automate the repetitive data update processes in the production database on desktop<br>
**Methods:**<br>
- Update trip polygons to keep pace with changing regions<br>

**Resources:**<br>
- [WGS 84 Spherical Mercator (WKID 3857)](https://epsg.io/3857)

|Date|Editor|Changes|
|---|---|---|
|11/24/2023|L. Kaufmann|File created|
||||

In [10]:
# IMPORT PACKAGES
import arcpy
import os

import logging
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

import datetime as dt
today = dt.datetime.now()

# ABOUT THE GEODATABASES
sr = arcpy.SpatialReference(3857)
arcpy.env.overwriteOutput = True

## Production
dbPROD_fldr = r"C:\Users\Laura\AppData\Roaming\Esri\ArcGISPro\Favorites\Travel_PROD.sde"
dbPROD_name = r"Travel_PROD.DBO"
dbPROD = os.path.join(dbPROD_fldr, dbPROD_name)

## Development
dbDEV_fldr = r"C:\Users\Laura\AppData\Roaming\Esri\ArcGISPro\Favorites\Travel_DEV.sde"
dbDEV_name = r"Travel_DEV.DBO"
dbDEV = os.path.join(dbDEV_fldr, dbDEV_name)

## Scratch
dbScratch = r"C:\Users\Laura\Documents\Keepsakes\Travel\TravelMaster\Scratch.gdb"

logging.info('Packages imported; ready to begin')

2023-11-26 20:26:03,105 - Packages imported; ready to begin


## Update Trip polygons
The script below updates 

**Resources:**<br>
- [Feature Compare](https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/feature-compare.htm): Compares two feature classes or layers and returns the comparison results

In [11]:
sourceDB = r"C:\Users\Laura\Documents\Keepsakes\Travel\0_MetadataInstructions\2022 Database Migration\Travel_Archive.gdb"

regions = os.path.join(sourceDB, "Regions")
trips = os.path.join(sourceDB, "Trips")
worldhex = os.path.join(sourceDB, "WorldHex15000")

euro_poly_fc_name = os.path.join(dbScratch, "Dissovled_Trips")

logging.info('Data inputs defined')

# Create layer of dissolved hexagons

try:
    ## Select features where Trip_ID <> "000"
    hex_europe = arcpy.management.SelectLayerByAttribute(worldhex, "NEW_SELECTION", "Trip_ID IS NOT NULL", None)

    ## Generate merged shapes based on the Trip_ID
    dissolved_trips = arcpy.analysis.PairwiseDissolve(hex_europe, euro_poly_fc_name, ["Trip_ID"], "", "SINGLE_PART")
    
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=hex_europe,
        selection_type="CLEAR_SELECTION",
        where_clause="",
        invert_where_clause=None
    )
    logging.info('WorldHex features dissolved')
    
except:
    logging.info('Something went wrong dissolving WorldHex features. Please check the script')

# Create list of Trip_IDs from the Trips layer

try:
    ## Define empty list
    trip_ids = []

    ## Use a search cursor to populate the list
    with arcpy.da.SearchCursor(trips,["Trip_ID"]) as id_search:
        for row in id_search:
            trip_ids.append(row[0])
            
    logging.info('Populating the list of Trip_IDs from Trips')

except:
    logging.info('Something went wrong populating the list of Trip_IDs. Please check the script')
    
finally:
    del id_search

2023-11-26 20:26:07,932 - Data inputs defined
2023-11-26 20:26:10,094 - WorldHex features dissolved
2023-11-26 20:26:10,112 - Populating the list of Trip_IDs from Trips


In [18]:
# Decide whether a Trip polygon needs to be updated and write relevant vertices to a dictionary

try:
    ## Define empty dictionary
    new_shape_dict = {}

    ## Iterate through list of trips defined in previous try/except block
    for tid in trip_ids:
        ### Define variables
        id_qry = str("Trip_ID = '" + tid + "'")

        hex_selection = arcpy.management.SelectLayerByAttribute(dissolved_trips, "NEW_SELECTION", id_qry, None)
        hex_count = str(arcpy.management.GetCount(hex_selection))
        hex_count = int(hex_count)

        trip_selection = arcpy.management.SelectLayerByAttribute(trips, "NEW_SELECTION", id_qry, None)
        
        #### Test to ensure that WorldHex dissolved into one polygon per Trip
        if hex_count > 1:
            logging.info('%s has more than one polygon attributed to it. Please review the WorldHex feature class to ensure the selected hexagons will dissolve into a single polygon.', tid)
        elif hex_count < 1:
            logging.info("No hexagons have been assigned to %s. Please review the Trips and WorldHex feature classes to ensure alignment on the Trip_ID field.", tid)
        else:
            ##### Define variables
            trips_compare = os.path.join(dbScratch, "\Trips_Compare_{0}".format(tid))
            
            shape_compare = arcpy.management.FeatureCompare(hex_selection, trip_selection, ["Trip_ID",], "GEOMETRY_ONLY", "IGNORE_M;IGNORE_Z;IGNORE_POINTID;IGNORE_EXTENSION_PROPERTIES;IGNORE_SUBTYPES;IGNORE_RELATIONSHIPCLASSES;IGNORE_REPRESENTATIONCLASSES;IGNORE_FIELDALIAS", "0.001 Meters", 0.001, 0.001, None, None, "CONTINUE_COMPARE", trips_compare)
            compare_result = str(shape_compare[1])
            print("{0}: {1}".format(tid, compare_result))
        
            ##### Test if the geometry has changed
            if compare_result != str("false"):
                logging.info("%s geometry has not changed and will not be updated.", tid)
            else:
                ###### For features needing updates, create array of vertices
                pt_array = arcpy.da.FeatureClassToNumPyArray(hex_selection, ["Trip_ID", "Shape@XY"], explode_to_points=True)

                ###### Write array to empty dictionary
                for Trip_ID, xy in pt_array:
                    xy_tup = tuple(xy)

                    if Trip_ID not in new_shape_dict:
                        new_shape_dict[Trip_ID] = [xy_tup]
                    elif xy_tup not in new_shape_dict[Trip_ID]:
                        new_shape_dict[Trip_ID].append(xy_tup)
                        
    logging.info("Geometries compared and updates written to dictionary...")

except:
    logging.info("Something went wrong comparing the old and new Trip polygons. Please check the script.")

{4C735A2D-2230-4329-B7F4-1B01E0CD4B4C}: false
{BC69B63C-5B06-4608-90BA-7B1870F53E46}: false
{ECE0B424-1629-4EB8-99BD-D3A11BCCDBE6}: false
{F6A9B4CC-1B1B-4AC0-90D2-86319C1EAE8E}: false
{AF45280C-DA3B-4B09-B4B4-AA74247E4CF1}: false
{C9F200D2-432D-4EA8-90E4-BABFE1899368}: false
{C408474E-DCC9-41D7-BBC6-18A5DE2DC2C2}: false
{6BE1B146-6FB2-4418-AD94-75BA5D50C3C5}: false
{FD9F6638-EFD1-4D63-9E82-36123315FE37}: false
{E2464906-0E05-4632-978A-69A621D1D79F}: false
{B84E7FEB-87B8-413F-A30C-33681912AAEA}: false
{8391D175-4AA6-43E0-BC0D-F41053F04E6B}: false
{9FB34997-8C8D-4B68-B90C-D2E96A017134}: false
{00FCC36F-3E1C-44E0-A7BA-9F6FD4789564}: false
{915AAC29-FCEA-4724-9413-875BE6E3A007}: false


2023-11-26 20:35:46,092 - Geometries compared and updates written to dictionary...


In [17]:
print(new_shape_dict)

{'{4C735A2D-2230-4329-B7F4-1B01E0CD4B4C}': [(2197755.3407000005, 5246236.515100002), (2136613.490600001, 5140335.724299997), (2014329.7903999984, 5140335.724299997), (1953187.9404000007, 5246236.515100002), (1830904.2402000017, 5246236.515100002), (1769762.3900999986, 5352137.306000002), (1830904.2402000017, 5458038.0967999995), (1769762.3900999986, 5563938.887599997), (1647478.6900000013, 5563938.887599997), (1586336.8398999982, 5669839.678400002), (1464053.1396999992, 5669839.678400002), (1402911.2895999998, 5775740.4692), (1464053.1396999992, 5881641.259999998), (1402911.2895999998, 5987542.050800003), (1280627.5894999988, 5987542.050800003), (1219485.7393999994, 6093442.841600001), (1280627.5894999988, 6199343.632399999), (1402911.2895999998, 6199343.632399999), (1464053.1396999992, 6093442.841600001), (1586336.8398999982, 6093442.841600001), (1647478.6900000013, 5987542.050800003), (1769762.3900999986, 5987542.050800003), (1830904.2402000017, 5881641.259999998), (1953187.940400000

In [19]:
with arcpy.da.UpdateCursor(trips,["Trip_ID", "SHAPE@XY"]) as update_polygon:
    for row in update_polygon:
        record_id = row[0]
        if record_id in new_shape_dict:
            row[1] = new_shape_dict[record_id]
            print(row[1])
            update_polygon.updateRow(row)

[(2197755.3407000005, 5246236.515100002), (2136613.490600001, 5140335.724299997), (2014329.7903999984, 5140335.724299997), (1953187.9404000007, 5246236.515100002), (1830904.2402000017, 5246236.515100002), (1769762.3900999986, 5352137.306000002), (1830904.2402000017, 5458038.0967999995), (1769762.3900999986, 5563938.887599997), (1647478.6900000013, 5563938.887599997), (1586336.8398999982, 5669839.678400002), (1464053.1396999992, 5669839.678400002), (1402911.2895999998, 5775740.4692), (1464053.1396999992, 5881641.259999998), (1402911.2895999998, 5987542.050800003), (1280627.5894999988, 5987542.050800003), (1219485.7393999994, 6093442.841600001), (1280627.5894999988, 6199343.632399999), (1402911.2895999998, 6199343.632399999), (1464053.1396999992, 6093442.841600001), (1586336.8398999982, 6093442.841600001), (1647478.6900000013, 5987542.050800003), (1769762.3900999986, 5987542.050800003), (1830904.2402000017, 5881641.259999998), (1953187.9404000007, 5881641.259999998), (2014329.7903999984,

TypeError: unsupported geometry type